In [ ]:
import pandas as pd
from pathlib import Path  

Create FRED Economic Data specific function for formating/cleaning csv files

In [ ]:
def fred_clean(name, file):
    """
    Preprocesses CSV FRED economic data for missing values and reindexes
    datatime values. Returns a pandas dataframe.

    Args:
        name (str): Names the dataframe and title of value header column
        file (str): File name of downloaded FRED data set

    Returns:
        pandas.core.frame.DataFrame: processed dataframe
    """
    
    df = str(name) + '_df'
    df = pd.read_csv(r'./data/' + file)
    # rename header, formate datetime, remove duplicates, reindex datetime
    df.columns = ['date', name]
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.strftime('%Y-%m')
    df = df.drop_duplicates('date')
    df = df.set_index('date', drop = False)
    df = df.sort_index()
    df = df.loc['2003-01': '2022-12']
    
    # replace values in this dataset that are str('.') 
    df = df.replace('.', 0)
        
    # change data type to float in cpi columns
    df[[name]] = df[[name]].astype(float)
    df['date'] = pd.to_datetime(df['date'])
    
    return df

CREATE: df_11 (government economic data)

In [ ]:
# import FRED government economic data
m1_supply_df = fred_clean('m1_supply', 'M1SL.csv')
gdp_df = fred_clean('gdp', 'GDP.CSV')
fed_funds_rate_df = fred_clean('fed_funds_rate', 'FEDFUNDS.csv')
fed_bal_df = fred_clean('fed_bal', 'RESPPANWW.csv')
gov_debt_df = fred_clean('gov_debt', 'GFDEBTN.csv')

In [ ]:
# remove date column (currently indexed)
m1_supply_df = m1_supply_df.drop(['date'], axis=1)
gdp_df = gdp_df.drop(['date'], axis=1)
fed_funds_rate_df = fed_funds_rate_df.drop(['date'], axis=1)
fed_bal_df = fed_bal_df.drop(['date'], axis=1)
gov_debt_df = gov_debt_df.drop(['date'], axis=1)

In [ ]:
# merge all data frames to df_11
df_11 = pd.merge(m1_supply_df, gdp_df, how='outer', on = 'date')\
    .merge(fed_funds_rate_df, how='outer', on = 'date')\
        .merge(fed_bal_df, how='outer', on = 'date')\
            .merge(gov_debt_df, how='outer', on = 'date')

#linear interpolation for government_debt_usd quarterly data
df_11 = df_11.interpolate(limit=1, limit_direction='both').fillna(method='ffill')

#create velocity_money = gdp / m1_supply
df_11 ['velocity_money'] = df_11['gdp'] / df_11['m1_supply']
df_11

In [ ]:
# save data frame as df_11 
filepath = Path('./dataframes/df_11.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_11.to_csv(filepath)

CREATE: df_12 (asset pricing data)

In [ ]:
# import FRED asset pricing data
nasdaq100_df = fred_clean('nasdaq100', 'NASDAQ100.csv')
btc_price_df = fred_clean('btc_price', 'CBBTCUSD.csv')
housing_index_df = fred_clean('housing_index', 'USSTHPI.csv')
commod_index_df = fred_clean('commod_index', 'PALLFNFINDEXM.csv')
energy_index_df = fred_clean('energy_index', 'PNRGINDEXM.csv')

In [ ]:
# drop all other rows after first of month
nasdaq100_df = nasdaq100_df.assign(M=nasdaq100_df["date"]\
    .dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
btc_price_df = btc_price_df.assign(M=btc_price_df["date"]\
    .dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
housing_index_df = housing_index_df.assign(M=housing_index_df["date"]\
    .dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
commod_index_df = commod_index_df.assign(M=commod_index_df["date"]\
    .dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
energy_index_df = energy_index_df.assign(M=energy_index_df["date"]\
    .dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)

In [ ]:
# remove date column (currently indexed)
nasdaq100_df = nasdaq100_df.drop(['date'], axis=1)
btc_price_df = btc_price_df.drop(['date'], axis=1)
housing_index_df = housing_index_df.drop(['date'], axis=1)
commod_index_df = commod_index_df.drop(['date'], axis=1)
energy_index_df = energy_index_df.drop(['date'], axis=1)

In [ ]:
# merge all data frames to df_12
df_12 = pd.merge(nasdaq100_df, btc_price_df, how='outer', on = 'date')\
    .merge(housing_index_df, how='outer', on = 'date')\
        .merge(commod_index_df, how='outer', on = 'date')\
            .merge(energy_index_df, how='outer', on = 'date')

# linear interpolation for government_debt_usd quarterly data
df_12 = df_12.interpolate(limit=1, limit_direction='both').fillna(method='ffill')
df_12

In [ ]:
# save data frame as df_12 
filepath = Path('./dataframes/df_12.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_12.to_csv(filepath)

CREATE: df_13 (consumer expenses data)

In [ ]:
# import FRED consumer expenses data
cpi_index_df = fred_clean('cpi_index', 'CPIAUCSL.csv')
unit_labor_df = fred_clean('unit_labor', 'ULCNFB.csv')
personal_savings_df = fred_clean('personal_savings','PSAVE.csv')
personal_expenses_df = fred_clean('personal_expenses', 'PCE.csv')
rent_prim_res_df = fred_clean('rent_prim_res', 'CUUR0000SEHA.csv')

In [ ]:
# remove date column (currently indexed)
cpi_index_df = cpi_index_df.drop(['date'], axis=1)
unit_labor_df = unit_labor_df.drop(['date'], axis=1)
personal_savings_df = personal_savings_df.drop(['date'], axis=1)
personal_expenses_df = personal_expenses_df.drop(['date'], axis=1)
rent_prim_res_df = rent_prim_res_df.drop(['date'], axis=1)

In [ ]:
# merge all data frames to df_13
df_13 = pd.merge(cpi_index_df, unit_labor_df, how='outer', on = 'date')\
    .merge(personal_savings_df, how='outer', on = 'date')\
        .merge(personal_expenses_df, how='outer', on = 'date')\
            .merge(rent_prim_res_df, how='outer', on = 'date')

# linear interpolation for government_debt_usd quarterly data
df_13 = df_13.interpolate(limit=1, limit_direction='both').fillna(method='ffill')
df_13

In [ ]:
# save data frame as df_13
filepath = Path('./dataframes/df_13.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_13.to_csv(filepath)

Merge df_11, df_12 and df_13 to df_1 (Final Dataframe)

In [ ]:
# merge df_11, df_12, df_13
df_1 = pd.merge(df_11, df_12, how='outer', on = 'date')\
    .merge(df_13, how='outer', on = 'date')
df_1

In [ ]:
# save data frame as df_1
filepath = Path('./dataframes/df_1.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_1.to_csv(filepath)